# Tweet Creator

In [ ]:
import openai
import os 
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = ChatOpenAI(openai_api_key=openai.api_key,model_name="gpt-3.5-turbo")

In [ ]:
class My_List(BaseModel):
    name: str = Field(description="")
    list_items: list = Field(description="")

def list_creator(topic):
    My_List.name=f"Name of the list of the {topic}"
    My_List.list_items=f"A list of the {topic}"
    
    parser = PydanticOutputParser(pydantic_object=My_List)
    
    human_text = "{instruction}\n{format_instructions}"
    human_prompt = HumanMessagePromptTemplate.from_template(human_text)
    
    chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

    prompt = chat_prompt.format_prompt(instruction=f"Create a list for {topic}",
    format_instructions=parser.get_format_instructions()).to_messages()
    
    reply = llm(prompt)
     
    return parser.parse(reply.content)

def tweet_creator(topic, tone):
    system_template = 'You are a social media expert.'
    system_message = SystemMessagePromptTemplate.from_template(system_template)
    
    human_template = "Write a tweet about {topic}.  The tone should be {tone}.  The tweet should be no longer than 240 characters."
    human_message = HumanMessagePromptTemplate.from_template(human_template)
    
    chat_prompt = ChatPromptTemplate.from_messages([system_message,human_message])
    prompt = chat_prompt.format_prompt(topic=topic,tone=tone).to_messages()
    reply = llm(prompt)

    return reply.content

    
topic = input("Please enter a topic for a list: ")
result_list = list_creator(topic)
print("------------")
for i, result in enumerate(result_list.list_items, start=1):
    print(f"{i}. {result}")
    
topic_number = int(input("Enter the topic number to make a tweet about it."))    
tone = input("Enter the tone for the tweet.")
tweet = tweet_creator(result_list.list_items[topic_number -1], tone)
print("------------")
print(f"The tweet: {tweet}")

